In [ ]:
EC_size_list_iter = []#[10, 20, 50]  # List of sizes of the EC to test in iter mode
EC_size_list_enum = [3,4] #[5, 10, 20]  # List of sizes of the EC to test in enum mode

In [3]:
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using Games
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings
using YAML

In [ ]:
list_enums = Dict(
    size_enum=>load("$parent_dir/results_paper/enum/enum_simulations_results_$EC_enum_s.jld2")
    for size_enum in EC_size_list_enum
)